## Empty server

In [4]:
from nanover.omni import OmniRunner

runner = OmniRunner.with_basic_server()

## Client connected to empty server

In [6]:
from nanover.websocket import NanoverImdClient

client = NanoverImdClient.from_runner(runner)

## Simulation

In [2]:
from nanover.omni.openmm import OpenMMSimulation

input_xml = "openmm_files/nanotube.xml"
nanotube_simulation = OpenMMSimulation.from_xml_path(input_xml)

## Publish simulation frames via client

In [9]:
from contextlib import suppress
from nanover.app import NanoverImdApplication
from nanover.utilities.timing import yield_interval
from nanover.trajectory.convert import pack_dict_frame

fake_app_server = NanoverImdApplication.null_server()

def send_frame(frame_index, frame):
    frame.frame_index = frame_index
    message = {
        "frame": pack_dict_frame(frame.frame_dict),
    }
    client.send_message(message)

nanotube_simulation.load()
nanotube_simulation.reset(fake_app_server)
frame = nanotube_simulation.make_topology_frame()
send_frame(0, frame)

with suppress(KeyboardInterrupt):
    for dt in yield_interval(1/30):
        nanotube_simulation.advance_by_one_step()
        frame = nanotube_simulation.make_regular_frame()
        send_frame(1, frame)